In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import re

import copy

import numpy as np
import pandas as pd
import xarray as xr
xr.set_options(display_expand_data=False)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook',palette='bright',style='ticks')

import tqdm

# import hvplot.xarray
# import holoviews as hv
# hv.extension('bokeh')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.figure_formats = ['svg']

In [ ]:
from AFL.automation import prepare
from AFL.automation.shared.units import units,is_concentration,is_volume,is_mass
from AFL.automation.APIServer.Client import Client

# Deck

In [ ]:
deck =  prepare.Deck()
tipracks=[(3,'opentrons_96_tiprack_300ul'),(6,'opentrons_96_tiprack_300ul')]
deck.add_pipette('p300_single_gen2','left',tipracks=tipracks)

tipracks=[(1,'opentrons_96_tiprack_1000ul'),(9,'opentrons_96_tiprack_1000ul')]
deck.add_pipette('p1000_single_gen2','right',tipracks=tipracks)

deck.add_container('nist_6_20ml_stirred_vials',5)

deck.add_container('shaker_nest_96_wellplate_1600ul',4)

catch_loc = '10A1'
deck.add_catch ('nist_pneumatic_loader',10)

In [ ]:
deck.init_remote_connection('piot2',home=True)
deck.client.enqueue(task_name="reset_tipracks")
deck.send_deck_config()

In [ ]:
deck.make_align_script('/home/afl642/align.py')

# Make Stocks and Add to Deck

In [ ]:
prepare.db.read()
prepare.db

In [ ]:
deck.reset_stocks()

stock_H2O = prepare.Solution('stock_H2O',['H2O'])
stock_H2O.volume = 20*units("ml")
deck.add_stock(stock_H2O,'5B1')

stock_D2O = prepare.Solution('stock_D2O',['D2O'])
stock_D2O.volume = 20*units("ml")
deck.add_stock(stock_D2O,'5A1')

latex = 'latex_A66'
phi_latex = 0.472 #A66
stock_latex = prepare.Solution('stock_latex',['H2O',latex])
stock_latex.volume = 20*units("ml")
stock_latex.volume_fraction = {latex:phi_latex}
deck.add_stock(stock_latex,'5B3')


pigment = 'TiO2_R746'
wtfrac_slurry = 0.765#R746
stock_TiO2 = prepare.Solution('stock_TiO2',['H2O',pigment])
stock_TiO2.volume = 20*units("ml")
stock_TiO2.mass_fraction = {pigment:wtfrac_slurry}
deck.add_stock(stock_TiO2,'5B2')

# Mask Calculation

In [ ]:
v_total = 900*units('ul')
PVC=0.8
phi_T=0.20

target_vfrac =  {'H2O': 0.5291556678471582 ,'D2O': 0.4708443321528418 }
deck.reset_targets()
for PVC in np.linspace(0.0,1.0,9):
    for phi_T in np.linspace(0.025,0.25,9):

        target = prepare.Solution('target',['H2O','D2O',latex,pigment])
        target[latex].volume = v_total * (phi_T/phi_latex) * (1-PVC)
        target[pigment].volume = v_total * (phi_T/phi_slurry) * PVC
        vHD2O = v_total - target[pigment].volume - target[latex].volume
        target['H2O'].volume = vHD2O*target_vfrac['H2O']
        target['D2O'].volume = vHD2O*target_vfrac['D2O']
        
        
        name=f'{latex}_{pigment}_phiT_{phi_T:4.3f}_PVC_{PVC:4.3f}'
        deck.add_target(target,name=name)
    
deck.volume_cutoff = 20*units('ul')
deck.make_sample_series(reset_sample_series=True)
deck.validate_sample_series(tolerance=0.15)

In [ ]:
x = []; y=[]
for sample,validated in deck.sample_series:
    if not validated:
        continue
    x.append(sample.target_check.concentration[pigment].to('mg/ml').magnitude)
    y.append(sample.target_check.concentration[latex].to('mg/ml').magnitude)
plt.scatter(x,y) 
    

# Calculate Mask

In [ ]:
components = ['TiO2','latex','H2O','D2O']
AL_components = ['TiO2','latex']
AL_components_grid = [k+'_grid' for k in AL_components]

In [ ]:
MB = deck.make_mass_balance()
MB.process_components()
samples = MB.sample_composition_space(grid_density=9,pipette_min=5*units('ul'),composition_data='conc')
# samples1 = MB.constrain_samples_conc(
#     {
#         'PS80':{'value':75*units('mg/ml'), 'compare':'lt'},
#         'benzyl_alcohol_solute':{'value':30*units('mg/ml'), 'compare':'lt'},
#         'phenol_solute':{'value':30*units('mg/ml'), 'compare':'lt'},
#     },
#     rtol=0.05)

MB.stock_samples.attrs['components'] = AL_components
MB.stock_samples.attrs['TiO2_range'] = [1,50]
MB.stock_samples.attrs['benzyl_alcohol_solute_range'] = [0,25]
MB.stock_samples.attrs['phenol_solute_range'] = [0,25]
MB.stock_samples.attrs['PS80_units'] = 'mg/ml'
MB.stock_samples.attrs['benzyl_alcohol_solute_units'] = 'mg/ml'
MB.stock_samples.attrs['phenol_solute_units'] = 'mg/ml'

In [ ]:
MB.calculate_bounds(components=AL_components,exclude_comps_below=0.0001,composition_data='samples_conc',ternary=False)

In [ ]:
MB.make_grid_mask(pts_per_row=35,ternary=False)
MB.stock_samples = MB.stock_samples.reset_index('grid').reset_coords().set_index(grid='grid_mask').sel(grid=True).reset_index('grid').reset_coords().rename({'grid_':'mask'})
MB.stock_samples

In [ ]:
%matplotlib tk
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')

grid_data = MB.stock_samples[['PS80_grid','benzyl_alcohol_solute_grid','phenol_solute_grid']]
mask = np.zeros_like(grid_data.PS80_grid,dtype=bool)
for z in np.linspace(2,50,25):
    mask = np.logical_or(mask,np.isclose(grid_data.PS80_grid,z,rtol=0.1))

    
grid_data = grid_data.where(grid_data.PS80_grid.copy(data=mask),drop=True)
z = grid_data.PS80_grid
x = grid_data.benzyl_alcohol_solute_grid
y = grid_data.phenol_solute_grid
ax.scatter(xs=x,ys=y,zs=z,color='g',marker='o')




In [ ]:
grid_mask = MB.stock_samples[['mask']+AL_components_grid]
grid_mask.attrs.update(MB.stock_samples.attrs)
grid_mask

In [ ]:
AL_kwargs = copy.copy(MB.stock_samples.attrs)
del AL_kwargs['hull']
del AL_kwargs['delaunay']
AL_kwargs['GP_domain_transform'] = 'range_scaled'
AL_kwargs

## Specify Mixing Wells in Wellplates

In [ ]:
w = prepare.make_wellplate_locs(slot=4,size=96)
deck.client.enqueue(task_name='add_prep_targets', targets=w, reset=True )

In [ ]:
deck.client.enqueue(task_name='set_shake',rpm=1000)

# Configure Sample Server

In [ ]:
sample_client = Client('localhost',port=5000)
sample_client.login('nb')

In [ ]:
catch_protocol = {'source': 'target',
 'dest': '10A1',
 'volume': 900.0,
 'mix_before': None,
 'mix_after': None,
 'blow_out': False,
 'aspirate_rate': 200,
 'dispense_rate': 200,
 'post_aspirate_delay': 5,
 'post_dispense_delay': 5}

sample_client.enqueue(task_name='set_catch_protocol',**catch_protocol)

In [ ]:
#sample_client.set_object(deck=deck)
sample_client.enqueue(task_name='reset_deck')
for stock in deck.stocks:
    sample_client.enqueue(
        task_name='add_stock',
        stock_dict=stock.to_dict(),
        loc=deck.stock_location[stock],
    )
    
for mount,(name,tipracks) in deck.pipettes.items(): 
    tipracks = [(t,deck.tip_racks[t]) for t in tipracks]
    sample_client.enqueue(
        task_name='add_pipette',
        name=name,
        mount=mount,
        tipracks=tipracks,
    )
    
for slot,container in deck.containers.items(): 
    sample_client.enqueue(
        task_name='add_container',
        name=container,
        slot=slot
    )
    
for slot,catch in deck.catches.items(): 
    sample_client.enqueue(
        task_name='add_catch',
        name=catch,
        slot=slot
    )

In [ ]:
mix_order =  [
    'stock_H2O',
    'stock_D2O',
    'stock_TiO2',
    'stock_latex',
]

custom_stock_settings = {
    #example: '1A1': {'post_aspirate_delay':2,
    #                 'post_dispense_delay':2,
    #                 'aspirate_rate':40,
    #                 'dispense_rate':40
    #                 },
    
    deck.get_stock('stock_H2O')[1]: {'post_aspirate_delay':1,
                    'post_dispense_delay':1,
                    'aspirate_rate':200,
                    'dispense_rate':200
                    }  , 
    deck.get_stock('stock_D2O')[1]: {'post_aspirate_delay':1,
                    'post_dispense_delay':1,
                    'aspirate_rate':200,
                    'dispense_rate':200
                    }  , 
    deck.get_stock('stock_TiO2')[1]: {'post_aspirate_delay':10,
                     'post_dispense_delay':10,
                     'aspirate_rate':50,
                     'dispense_rate':50
                     }  , 
    deck.get_stock('stock_latex')[1]: {'post_aspirate_delay':5,
                     'post_dispense_delay':5,
                     'aspirate_rate':100,
                     'dispense_rate':100
                     }  , 
 
}
custom_stock_settings

In [ ]:
client_dict = {
    'prep':'piot2:5000',
    'load':'piloader:5000',
    'agent':'localhost:5053',
    'inst':'130.246.38.36:5000',
}
instrument = [
    {
        'name':'ISISLarmor',
        'client_name':'inst',
        'load_dest_label':'afterSPEC',
        'empty_base_kw': {'task_name':'expose', 'exposure':1.0, 'exposure_trans':0.5, 'expose_metric':'uamps','reduce_data':True},
        'measure_base_kw': {'task_name':'expose', 'exposure':5.0, 'exposure_trans':2.0, 'expose_metric':'uamps','reduce_data':True},
                            
    }
]
sample_client.set_config(
    data_path='/home/afl642/LARMOR_data/',
    client=client_dict,
    instrument=instrument,
    ternary=False,
    data_tag='tio2_latex_series1',
    components=[pigment,latex,'H2O','D2O'],
    custom_stock_settings=custom_stock_settings,
    mix_order=mix_order
    
)

In [ ]:
deck.sample_series

# Enqueue Series

In [ ]:
for sample,validated in deck.sample_series:
    if not validated:
        continue
    composition = {k:{'value':v.magnitude ,'units':str(v.units)} for k,v in  sample.target_check.concentration.items()}
    
    task = {}
    task['task_name'] = 'process_sample'
    task['composition'] = composition
    task['sample_volume'] = {'value':900,'units':'ul'}
    task['fixed_concs'] = []
    task['predict_next'] = False
    task['enqueue_next'] = False
    task['name'] = sample.name
    
    sample_client.enqueue(interactive=False,**task)

In [ ]:
sample

# Loader

In [ ]:
load = Client("piloader",port=5000)
load.login('nb')

In [ ]:
load.enqueue(task_name='rinseCell')

In [ ]:
load.enqueue(task_name='calibrate_sensor')

In [ ]:
load.enqueue(task_name='loadSample',load_dest_label='afterSPEC')

In [ ]:
load.enqueue(task_name='advanceSample',load_dest_label='afterSANS')

In [ ]:
load.get_config("all",interactive=True)

In [ ]:
rinse_program = [ 
    ['rinse1', 30], 
    [None, 5], 
    ['rinse2', 30], 
    [None, 5],
   ['blow', 20],
   [None, 5],
   ['blow', 5],
   [None, 5],
   ['blow', 5],
   [None, 5],
   ['blow', 10],
   [None, 5],
   ['blow', 10],
   [None, 5],
   ['blow', 10]
]
load.set_config(rinse_program=rinse_program)

In [ ]:
rinse_program = [  
    ['rinse1', 5], 
    [None,5], 
    ['rinse2', 5], 
    [None,5], 
    ['blow', 5]
]
load.set_config(rinse_program=rinse_program)

# Larmor

In [ ]:
client = Client('130.246.38.36',port=5000)
client.login('test')
client.pause(False)

In [ ]:
client.enqueue(
    task_name='expose',
    name='{Empty Cell PostWindowReplace}',
    exposure=10, 
    exposure_trans=2,
    expose_metric='uamps', 
    reduce_data=True
)

# sdfa

# Analyze Series

In [4]:
from tiled.client import from_uri
from tiled.queries import Eq,Contains, NotIn, Regex

tiled_client = from_uri('http://localhost:8000',api_key='NistoRoboto642')
result = (
    tiled_client
    .search(Eq('task_name','expose'))
    # .search(Eq('array_name','I'))
    .search(In('array_name',['I','err']))
    .search(Eq('driver_config.empty_cell_scatt_rn',80563))
    .search(Contains('name','micelles'))
)
result

<Container {}>

In [ ]:
ds_list = []

for uid,array in result.items():
    
    name = array.metadata['name']
    if 'MT-' in name:
        continue
    
        
    
    ds = xr.Dataset()
    ds["I"] = xr.DataArray(array[()],dims=['q'],coords={'q':array.metadata['q']})
    ds["name"] = name
    ds['dI']  
    #ds["phi"] = phi
    ds_list.append(ds)

ds = xr.concat(ds_list,dim='sample')
ds.to_netcdf('~/Documents/brads_data/5component_salt_micelle_tiled_pull.nc')
# lines = ds.set_index(sample='name').I.afl.scatt.plot_loglog(x='q')
# plt.legend(ds.name.values,bbox_to_anchor=(1.0,0.5),loc=6)

In [ ]:
plt.figure(figsize=(4,24))
ds.I.afl.scatt.plot_logwaterfall()

In [ ]:
tiled_client = from_uri('http://localhost:8000',api_key='NistoRoboto642')
result = (
    tiled_client
    .search(Eq('task_name','expose'))
    .search(Eq('array_name','I'))
    .search(Contains('name','micelle'))
)

In [4]:
from tiled.client import from_uri
from tiled.queries import Eq,Contains, NotIn, Regex
import xarray as xr

tiled_client = from_uri('http://localhost:8000',api_key='NistoRoboto642')
result = (
    tiled_client
    .search(Eq('task_name','expose'))
    .search(Eq('array_name','I'))
    .search(Contains('sample_name','CuCl2'))
)

ds_list = []

for uid,array in result.items():
    
    name = array.metadata['name']
    if 'MT-' in name:
        continue
    
        
    
    ds = xr.Dataset()
    ds["I"] = xr.DataArray(array[()],dims=['q'],coords={'q':array.metadata['q']})
    ds["name"] = name
    ds_list.append(ds)

ds = xr.concat(ds_list,dim='sample')

ds.to_netcdf('~/Documents/brads_data/salt_SDS_I_tiled_pull.nc')

result = (
    tiled_client
    .search(Eq('task_name','expose'))
    .search(Eq('array_name','dI'))
    .search(Contains('sample_name','CuCl2'))
)

ds_list = []

for uid,array in result.items():
    
    name = array.metadata['name']
    if 'MT-' in name:
        continue
    
        
    
    ds = xr.Dataset()
    ds["dI"] = xr.DataArray(array[()],dims=['q'],coords={'q':array.metadata['q']})
    ds["name"] = name
    #ds["phi"] = phi
    ds_list.append(ds)

ds = xr.concat(ds_list,dim='sample')

ds.to_netcdf('~/Documents/brads_data/salt_SDS_dI_tiled_pull.nc')

# result = (
#     tiled_client
#     .search(Eq('task_name','expose'))
#     .search(Eq('array_name','I'))
#     .search(Contains('sample_name','CuCl2'))
# )

# ds_list = []

# for uid,array in result.items():
    
#     name = array.metadata['name']
#     if 'MT-' in name:
        
#         ds = xr.Dataset()
#         ds["I"] = xr.DataArray(array[()],dims=['q'],coords={'q':array.metadata['q']})
#         ds["name"] = name
#         ds_list.append(ds)

# ds = xr.concat(ds_list,dim='sample')

# ds.to_netcdf('~/Documents/brads_data/salt_SDS_blank_I_tiled_pull.nc')

# result = (
#     tiled_client
#     .search(Eq('task_name','expose'))
#     .search(Eq('array_name','dI'))
#     .search(Contains('sample_name','SDS'))
# )

# ds_list = []

# for uid,array in result.items():
    
#     name = array.metadata['name']
#     if 'MT-' in name:
        
#         ds = xr.Dataset()
#         ds["I"] = xr.DataArray(array[()],dims=['q'],coords={'q':array.metadata['q']})
#         ds["name"] = name
#         ds_list.append(ds)

# ds = xr.concat(ds_list,dim='sample')

# ds.to_netcdf('~/Documents/brads_data/salt_SDS_blank_dI_tiled_pull.nc')


ValueError: must supply at least one object to concatenate